In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xlrd

In [2]:
df_excel = pd.read_excel('/Users/Jia/Downloads/data_skripsi/tb/data_tb.xlsx')

Convert Datetime (TB)

In [3]:
# Mengonversi kolom 'time' ke tipe data datetime
df_excel['time'] = pd.to_datetime(df_excel['time'])

# Menambahkan kolom 'month'
df_excel['month'] = df_excel['time'].dt.month

# Menambahkan kolom 'year'
df_excel['year'] = df_excel['time'].dt.year

In [4]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   KABKOT     180 non-null    object        
 1   month      180 non-null    int32         
 2   year       180 non-null    int32         
 3   tb         180 non-null    int64         
 4   longitude  180 non-null    float64       
 5   latitude   180 non-null    float64       
 6   time       180 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(1)
memory usage: 8.6+ KB


Merapihkan Data TB 2020-2023

In [5]:
import calendar

In [6]:
# Filter for years 2020-2022
df_tb = df_excel[(df_excel['year'] >= 2020) & (df_excel['year'] <= 2022)]

# Group tb_total by KABKOT, month, and time
tb_total = df_tb.groupby(['KABKOT', 'month', 'time'])['tb'].sum()
tb_total

KABKOT         month  time      
JAKARTA BARAT  1      2020-01-01    715
                      2021-01-01    329
                      2022-01-01    851
               2      2020-02-01    803
                      2021-02-01    326
                                   ... 
JAKARTA UTARA  11     2021-11-01    475
                      2022-11-01    685
               12     2020-12-01    201
                      2021-12-01    450
                      2022-12-01    671
Name: tb, Length: 180, dtype: int64

Merapihkan Data Kelembapan (rh) 2020-2023

In [8]:
df_rh = pd.read_csv(r'/Users/Jia/Downloads/data_skripsi/kelembaban/dissolved_result_rh_3years.csv')
df_rh = df_rh.drop(['Unnamed: 0'], axis=1)
print(df_rh.columns)

Index(['KABKOT', 'month', 'year', 'longitude', 'latitude', 'rh', 'time'], dtype='object')


In [9]:
# Filter for years 2020-2022
df_rh = df_rh[(df_rh['year'] >= 2020) & (df_rh['year'] <= 2022)]

# Group rh_mean by KABKOT, month, and time
rh_mean = df_rh.groupby(['KABKOT', 'month', 'time'])['rh'].mean()
rh_mean

KABKOT         month  time      
JAKARTA BARAT  1      2020-01-01    85.895006
                      2021-01-01    85.869056
                      2022-01-01    83.321066
               2      2020-02-01    86.136422
                      2021-02-01    86.575353
                                      ...    
JAKARTA UTARA  11     2021-11-01    81.303521
                      2022-11-01    82.045862
               12     2020-12-01    83.099855
                      2021-12-01    83.487948
                      2022-12-01    83.006588
Name: rh, Length: 180, dtype: float64

Merge Files

In [10]:
# Merge DataFrames on 'KABKOT', 'month', and 'time'
df_tb_rh = pd.merge(tb_total, rh_mean, on=['KABKOT', 'month'])
df_tb_rh

tb         rh
KABKOT        month                
JAKARTA BARAT 1      715  85.895006
              1      715  85.869056
              1      715  83.321066
              1      329  85.895006
              1      329  85.869056
...                  ...        ...
JAKARTA UTARA 12     450  83.487948
              12     450  83.006588
              12     671  83.099855
              12     671  83.487948
              12     671  83.006588

[540 rows x 2 columns]

In [11]:
df_tb_rh = df_tb_rh.groupby(['KABKOT', 'month']).agg({'tb':'sum', 'rh':'mean'}).reset_index()
# reset_index() untuk memisahkan kolom KABKOT dan month

In [12]:
df_tb_rh

KABKOT  month    tb         rh
0     JAKARTA BARAT      1  5685  85.028376
1     JAKARTA BARAT      2  4758  85.414108
2     JAKARTA BARAT      3  5376  83.325529
3     JAKARTA BARAT      4  4578  82.655781
4     JAKARTA BARAT      5  4125  82.746073
5     JAKARTA BARAT      6  4920  82.785458
6     JAKARTA BARAT      7  4272  79.048910
7     JAKARTA BARAT      8  4539  77.938904
8     JAKARTA BARAT      9  5070  78.294391
9     JAKARTA BARAT     10  5145  79.946286
10    JAKARTA BARAT     11  5427  81.601883
11    JAKARTA BARAT     12  5037  83.210277
12    JAKARTA PUSAT      1  5775  85.181366
13    JAKARTA PUSAT      2  4221  85.650218
14    JAKARTA PUSAT      3  4869  83.408437
15    JAKARTA PUSAT      4  3981  82.712284
16    JAKARTA PUSAT      5  3579  82.786839
17    JAKARTA PUSAT      6  4167  82.744098
18    JAKARTA PUSAT      7  3810  78.976054
19    JAKARTA PUSAT      8  4242  77.808512
20    JAKARTA PUSAT      9  4515  78.054423
21    JAKARTA PUSAT     10  4548  80.246812
22    JAKARTA PUSAT     11  4572  81.802081
23    JAKARTA PUSAT     12  4434  83.277494
24  JAKARTA SELATAN      1  5691  85.463425
25  JAKARTA SELATAN      2  4320  86.022837
26  JAKARTA SELATAN      3  5205  83.634839
27  JAKARTA SELATAN      4  4611  83.092473
28  JAKARTA SELATAN      5  3795  83.127833
29  JAKARTA SELATAN      6  4920  83.163488
30  JAKARTA SELATAN      7  4458  79.043243
31  JAKARTA SELATAN      8  4653  78.043638
32  JAKARTA SELATAN      9  5199  78.398763
33  JAKARTA SELATAN     10  4980  80.737401
34  JAKARTA SELATAN     11  5142  82.187734
35  JAKARTA SELATAN     12  5154  83.536903
36    JAKARTA TIMUR      1  7437  85.471600
37    JAKARTA TIMUR      2  5949  86.069003
38    JAKARTA TIMUR      3  7809  83.731551
39    JAKARTA TIMUR      4  7101  83.039264
40    JAKARTA TIMUR      5  5958  83.083798
41    JAKARTA TIMUR      6  7458  82.996820
42    JAKARTA TIMUR      7  6303  79.150760
43    JAKARTA TIMUR      8  6822  78.022256
44    JAKARTA TIMUR      9  7641  78.248007
45    JAKARTA TIMUR     10  7803  80.862735
46    JAKARTA TIMUR     11  7650  82.232803
47    JAKARTA TIMUR     12  7467  83.577681
48    JAKARTA UTARA      1  4599  84.981667
49    JAKARTA UTARA      2  3207  85.362686
50    JAKARTA UTARA      3  4041  83.286913
51    JAKARTA UTARA      4  3657  82.465826
52    JAKARTA UTARA      5  3195  82.622919
53    JAKARTA UTARA      6  3492  82.510811
54    JAKARTA UTARA      7  3246  79.250368
55    JAKARTA UTARA      8  3633  78.138836
56    JAKARTA UTARA      9  4038  78.286553
57    JAKARTA UTARA     10  3960  80.078123
58    JAKARTA UTARA     11  4065  81.541914
59    JAKARTA UTARA     12  3966  83.198130

In [13]:
df_tb_rh.shape

(60, 4)

Datetime edit for 'month'

In [14]:
import calendar

# Format for monthly graph
df_tb_rh['month'] = pd.to_datetime(df_tb_rh['month'], format='%m').dt.month_name()

# Set the ['month_name'] as an index
df_tb_rh = df_tb_rh.set_index('month').reset_index()

In [15]:
df_tb_rh

month           KABKOT    tb         rh
0     January    JAKARTA BARAT  5685  85.028376
1    February    JAKARTA BARAT  4758  85.414108
2       March    JAKARTA BARAT  5376  83.325529
3       April    JAKARTA BARAT  4578  82.655781
4         May    JAKARTA BARAT  4125  82.746073
5        June    JAKARTA BARAT  4920  82.785458
6        July    JAKARTA BARAT  4272  79.048910
7      August    JAKARTA BARAT  4539  77.938904
8   September    JAKARTA BARAT  5070  78.294391
9     October    JAKARTA BARAT  5145  79.946286
10   November    JAKARTA BARAT  5427  81.601883
11   December    JAKARTA BARAT  5037  83.210277
12    January    JAKARTA PUSAT  5775  85.181366
13   February    JAKARTA PUSAT  4221  85.650218
14      March    JAKARTA PUSAT  4869  83.408437
15      April    JAKARTA PUSAT  3981  82.712284
16        May    JAKARTA PUSAT  3579  82.786839
17       June    JAKARTA PUSAT  4167  82.744098
18       July    JAKARTA PUSAT  3810  78.976054
19     August    JAKARTA PUSAT  4242  77.808512
20  September    JAKARTA PUSAT  4515  78.054423
21    October    JAKARTA PUSAT  4548  80.246812
22   November    JAKARTA PUSAT  4572  81.802081
23   December    JAKARTA PUSAT  4434  83.277494
24    January  JAKARTA SELATAN  5691  85.463425
25   February  JAKARTA SELATAN  4320  86.022837
26      March  JAKARTA SELATAN  5205  83.634839
27      April  JAKARTA SELATAN  4611  83.092473
28        May  JAKARTA SELATAN  3795  83.127833
29       June  JAKARTA SELATAN  4920  83.163488
30       July  JAKARTA SELATAN  4458  79.043243
31     August  JAKARTA SELATAN  4653  78.043638
32  September  JAKARTA SELATAN  5199  78.398763
33    October  JAKARTA SELATAN  4980  80.737401
34   November  JAKARTA SELATAN  5142  82.187734
35   December  JAKARTA SELATAN  5154  83.536903
36    January    JAKARTA TIMUR  7437  85.471600
37   February    JAKARTA TIMUR  5949  86.069003
38      March    JAKARTA TIMUR  7809  83.731551
39      April    JAKARTA TIMUR  7101  83.039264
40        May    JAKARTA TIMUR  5958  83.083798
41       June    JAKARTA TIMUR  7458  82.996820
42       July    JAKARTA TIMUR  6303  79.150760
43     August    JAKARTA TIMUR  6822  78.022256
44  September    JAKARTA TIMUR  7641  78.248007
45    October    JAKARTA TIMUR  7803  80.862735
46   November    JAKARTA TIMUR  7650  82.232803
47   December    JAKARTA TIMUR  7467  83.577681
48    January    JAKARTA UTARA  4599  84.981667
49   February    JAKARTA UTARA  3207  85.362686
50      March    JAKARTA UTARA  4041  83.286913
51      April    JAKARTA UTARA  3657  82.465826
52        May    JAKARTA UTARA  3195  82.622919
53       June    JAKARTA UTARA  3492  82.510811
54       July    JAKARTA UTARA  3246  79.250368
55     August    JAKARTA UTARA  3633  78.138836
56  September    JAKARTA UTARA  4038  78.286553
57    October    JAKARTA UTARA  3960  80.078123
58   November    JAKARTA UTARA  4065  81.541914
59   December    JAKARTA UTARA  3966  83.198130

Overlay layer plot for TB and RH

In [16]:
indicators = ['tb', 'rh']

# Group data by KABKOT
grouped_data = df_tb_rh.groupby('KABKOT')

# Plot time series for 'tb' and 'so2' for each KABKOT
fig, axs = plt.subplots(len(grouped_data), 1, figsize=(20, len(grouped_data)*5), sharex=False, gridspec_kw={'hspace': 0.2})


for (name, group), ax in zip(grouped_data, axs):
    x = group['month']
    y1 = group['tb']
    y2 = group['rh']

    ax.plot(x, y1, label='Kejadian Tuberkulosis', marker='o', color='darkcyan')
    ax.set_ylabel(f'Jumlah Kejadian Tuberkulosis (jiwa)', color='darkcyan')
    ax.tick_params(axis='y', labelcolor='darkcyan')

    ax2 = ax.twinx()
    ax2.plot(x, y2, label='Curah Hujan', marker='o', color='brown')
    ax2.set_ylabel(f'Rata-Rata Kelembapan (%)', color='brown')
    ax2.tick_params(axis='y', labelcolor='brown')

    ax.legend(loc= 'upper left', fontsize=16)
    ax2.legend(loc= 'upper right', fontsize=16)
    
# Add common x-axis label
axs[-1].set_xlabel('Month', fontsize= 14)
axs[-1].set_title('JAKARTA UTARA', fontsize= 14)
axs[-2].set_title('JAKARTA TIMUR',fontsize= 14)
axs[-3].set_title('JAKARTA SELATAN', fontsize= 14)
axs[-4].set_title('JAKARTA PUSAT', fontsize= 14)
axs[-5].set_title('JAKARTA BARAT', fontsize= 14)

plt.show()